#### Loan Approval System

Author: Mohammed Iftkhar

Date: 29th March, 2025

##### Introduction
In the modern financial landscape, loan approval decisions play a crucial role in banking and credit systems. Traditionally, loan approval was based on manual assessment, leading to inefficiencies and potential biases. This project aims to build a Loan Approval System using machine learning (ML) and deep learning techniques to automate, optimize, and enhance the accuracy of loan approval decisions.

By leveraging data preprocessing, exploratory data analysis (EDA), feature engineering, feature selection, and multiple ML models, this project provides an intelligent loan approval system capable of making data-driven decisions.

##### Description
The project follows a systematic approach to data-driven loan approval using machine learning techniques:

1. Data Collection & Preprocessing
- Load the dataset, clean missing values, and standardize column names.
- Convert categorical variables into numerical values.
- Handle outliers using statistical techniques.

2. Exploratory Data Analysis (EDA)
- Perform statistical analysis and visualization to understand the dataset.
- Identify key patterns and relationships among loan approval factors.

3. Feature Engineering & Selection
- Engineer new features such as loan-to-income ratio and asset-to-loan ratio.
- Select the most important features using ANOVA F-test to enhance model performance.

4. Model Training & Evaluation

    ✅ Train multiple machine learning models, including:
- Logistic Regression
- Decision Tree
- Random Forest
- Support Vector Machine (SVM)
- K-Nearest Neighbors (KNN)
- Gradient Boosting

    ✅ Use SMOTE to handle class imbalance.

    ✅ Evaluate models using accuracy, precision, recall, F1-score, and ROC-AUC curves.

5. Neural Network Implementation
- Build and train a deep learning model using TensorFlow & Keras.
- Implement Batch Normalization, Dropout, and Early Stopping for optimization.

6. Model Selection & Deployment
- Select the best-performing model (Random Forest in this case).
- Save the trained model using joblib for future predictions.



In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from imblearn.over_sampling import SMOTE

In [1]:
# Scikit-learn libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
# Tensorflow libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

##### Configurations

In [3]:
warnings.filterwarnings("ignore")

# Set aesthetic parameters
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

NameError: name 'warnings' is not defined

##### Load Dataset

In [ ]:
# Load and inspect dataset
def load_data():
    df = pd.read_csv("data.csv")
    print("Dataset loaded successfully!")
    print(f"\nShape: {df.shape}")
    print("\nFirst 5 rows:")
    display(df.head())
    return df

##### Data Cleaning

In [ ]:
# Data Cleaning and Preprocessing
def clean_data(df):
    # Make column names consistent
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

    # Handle missing values
    print("\nMissing values before cleaning:")
    print(df.isnull().sum())

    # Drop rows with missing values (alternative: imputation could be used)
    df.dropna(inplace=True)

    # Convert data types
    numeric_cols = [
        "no_of_dependents",
        "income_annum",
        "loan_amount",
        "loan_term",
        "cibil_score",
        "residential_assets_value",
        "commercial_assets_value",
        "luxury_assets_value",
        "bank_asset_value",
    ]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("int64")

    print("\nMissing values after cleaning:")
    print(df.isnull().sum())

    return df

##### Exploratory Data Analysis

In [ ]:
def perform_eda(df):
    print("\nBasic Statistics:")
    display(df.describe())

    print("\nCategorical Features Distribution:")
    cat_cols = ["education", "self_employed", "loan_status"]
    for col in cat_cols:
        plt.figure(figsize=(8, 5))
        sns.countplot(data=df, x=col, palette="viridis")
        plt.title(f"Distribution of {col}")
        plt.xticks(rotation=45)
        plt.show()
        print(df[col].value_counts(normalize=True))

    # Correlation analysis
    plt.figure(figsize=(12, 8))
    corr = df.corr(numeric_only=True)
    sns.heatmap(corr, annot=True, cmap="coolwarm", center=0, fmt=".2f")
    plt.title("Feature Correlation Matrix")
    plt.show()

    # Identify highly correlated features
    high_corr = []
    for col in corr.columns:
        high_corr_count = (corr[col].abs() > 0.8).sum() - 1  # Exclude self-correlation
        if high_corr_count >= 1:
            high_corr.append(col)

    print("\nFeatures with high correlation (>0.8):", high_corr)

    # Visualize numeric features with dynamic layout
    numeric_features = df.select_dtypes(include=["int64"]).columns
    num_features = len(numeric_features)

    # Calculate optimal grid size
    cols = 3
    rows = (num_features // cols) + (1 if num_features % cols else 0)

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()  # Flatten in case it's a 2D array

    for i, col in enumerate(numeric_features):
        if i < len(axes):  # Ensure we don't exceed the number of subplots
            sns.histplot(data=df, x=col, bins=20, kde=True, ax=axes[i])
            axes[i].set_title(f"Distribution of {col}")

    # Hide any empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.suptitle("Distribution of Numeric Features", y=1.02)
    plt.tight_layout()
    plt.show()

    return df

In [ ]:
# Feature Engineering
def engineer_features(df):
    # Drop loan_id as it's just an identifier
    df.drop("loan_id", axis=1, inplace=True)

    # Create new features
    df["total_assets"] = (
        df["residential_assets_value"]
        + df["commercial_assets_value"]
        + df["luxury_assets_value"]
        + df["bank_asset_value"]
    )

    df["loan_to_income_ratio"] = df["loan_amount"] / df["income_annum"]
    df["asset_to_loan_ratio"] = df["total_assets"] / df["loan_amount"]

    # Encode categorical features
    le = LabelEncoder()
    df["education"] = le.fit_transform(df["education"])
    df["self_employed"] = le.fit_transform(df["self_employed"])
    df["loan_status"] = le.fit_transform(df["loan_status"])

    # Handle outliers using winsorization
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns
    for col in numeric_cols:
        q1 = df[col].quantile(0.05)
        q3 = df[col].quantile(0.95)
        df[col] = np.where(df[col] < q1, q1, df[col])
        df[col] = np.where(df[col] > q3, q3, df[col])

    return df

In [ ]:
# Feature Selection
def select_features(df):
    X = df.drop("loan_status", axis=1)
    y = df["loan_status"]

    # Feature selection using ANOVA F-value
    selector = SelectKBest(f_classif, k="all")
    selector.fit(X, y)

    # Plot feature importance
    plt.figure(figsize=(12, 6))
    feature_scores = pd.DataFrame({"Feature": X.columns, "Score": selector.scores_})
    feature_scores = feature_scores.sort_values("Score", ascending=False)

    sns.barplot(data=feature_scores, x="Score", y="Feature", palette="viridis")
    plt.title("Feature Importance Scores (ANOVA F-value)")
    plt.xlabel("F-value")
    plt.tight_layout()
    plt.show()

    # Select top features
    selected_features = feature_scores[feature_scores["Score"] > 10]["Feature"].tolist()
    print("\nSelected features:", selected_features)

    return df[selected_features + ["loan_status"]]

In [ ]:
# Model Training and Evaluation
def train_models(df):
    X = df.drop("loan_status", axis=1)
    y = df["loan_status"]

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # Handle class imbalance
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Define models
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "SVM": SVC(probability=True, random_state=42),
        "KNN": KNeighborsClassifier(),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    }

    # Train and evaluate models
    results = []
    for name, model in models.items():
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)
        y_prob = (
            model.predict_proba(X_test)[:, 1]
            if hasattr(model, "predict_proba")
            else [0] * len(y_test)
        )

        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_prob) if any(y_prob) else 0
        report = classification_report(y_test, y_pred, output_dict=True)

        results.append(
            {
                "Model": name,
                "Accuracy": accuracy,
                "Precision": report["weighted avg"]["precision"],
                "Recall": report["weighted avg"]["recall"],
                "F1-Score": report["weighted avg"]["f1-score"],
                "ROC AUC": roc_auc,
            }
        )

        # Plot confusion matrix
        plt.figure(figsize=(6, 5))
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=["Rejected", "Approved"],
            yticklabels=["Rejected", "Approved"],
        )
        plt.title(f"{name} Confusion Matrix")
        plt.ylabel("Actual")
        plt.xlabel("Predicted")
        plt.show()

        # Plot ROC curve if applicable
        if any(y_prob):
            fpr, tpr, _ = roc_curve(y_test, y_prob)
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.2f})")
            plt.plot([0, 1], [0, 1], "k--")
            plt.xlabel("False Positive Rate")
            plt.ylabel("True Positive Rate")
            plt.title("ROC Curve")
            plt.legend()
            plt.show()

    # Display results
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values("Accuracy", ascending=False)

    print("\nModel Performance Comparison:")
    display(
        results_df.style.background_gradient(
            cmap="viridis", subset=["Accuracy", "ROC AUC"]
        )
    )

    # Plot model comparison
    plt.figure(figsize=(12, 8))
    results_df.set_index("Model").plot(
        kind="bar",
        y=["Accuracy", "F1-Score", "ROC AUC"],
        color=["#3498db", "#2ecc71", "#9b59b6"],
    )
    plt.title("Model Performance Comparison")
    plt.ylabel("Score")
    plt.xticks(rotation=45)
    plt.ylim(0.5, 1.0)
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.show()

    return results_df, X_train, X_test, y_train, y_test, scaler

In [ ]:
# Neural Network Model
def build_nn(X_train, y_train, X_test, y_test):
    model = Sequential(
        [
            Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
            BatchNormalization(),
            Dropout(0.3),
            Dense(64, activation="relu"),
            BatchNormalization(),
            Dropout(0.3),
            Dense(32, activation="relu"),
            Dense(1, activation="sigmoid"),
        ]
    )

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC()],
    )

    early_stop = EarlyStopping(
        monitor="val_loss", patience=5, restore_best_weights=True
    )

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=1,
    )

    # Plot training history
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.title("Model Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epoch")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Model Loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend()

    plt.tight_layout()
    plt.show()

    # Evaluate model
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("\nNeural Network Classification Report:")
    print(classification_report(y_test, y_pred))

    return model

In [ ]:
# Main execution
def main():
    # Load and preprocess data
    df = load_data()
    df = clean_data(df)
    df = perform_eda(df)
    df = engineer_features(df)
    df = select_features(df)

    # Train traditional ML models
    results_df, X_train, X_test, y_train, y_test, scaler = train_models(df)

    # Train neural network
    # nn_model = build_nn(X_train, y_train, X_test, y_test)

    # Save best model (Random Forest in this case)
    best_model = RandomForestClassifier(random_state=42)
    best_model.fit(
        scaler.fit_transform(df.drop("loan_status", axis=1)), df["loan_status"]
    )

    import joblib

    joblib.dump(best_model, "loan_approval_model.pkl")
    joblib.dump(scaler, "scaler.pkl")
    print("\nBest model saved as 'loan_approval_model.pkl'")

In [ ]:
if __name__ == "__main__":
    main()

##### Conclusion
This project successfully demonstrates the effectiveness of machine learning and deep learning in loan approval automation. Through thorough data preprocessing, feature engineering, and model evaluation, the system achieves high accuracy and reliable performance in predicting loan approvals.

Among all the models tested, the Random Forest Classifier provided the best performance, making it the ideal choice for deployment. Additionally, the deep learning model showed promising results, indicating the potential for further improvements.

By automating loan approvals, this system can:

✅ Improve decision-making speed

✅ Enhance accuracy & reduce human bias

✅ Optimize risk assessment for banks & financial institutions

##### Future Work
Although this project achieves high accuracy, there is always room for improvement. Some areas for future enhancements include:

🔹 Hyperparameter Tuning: Further fine-tuning ML and deep learning models using techniques like Bayesian Optimization and Grid Search.

🔹 Explainability & Interpretability: Implement SHAP and LIME to interpret model decisions transparently.

🔹 Integration with Real-Time Systems: Deploy the model as a web application (Flask/Django) or integrate it into a banking system for real-time predictions.

🔹 Advanced Deep Learning Architectures: Explore more sophisticated models like LSTM (Long Short-Term Memory) for sequential data processing.

🔹 Improved Data Handling: Use automated data pipelines and integrate the system with real-world financial datasets.